<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/366_EFIA_Orchestrator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## Orchestration & Control

### How the Agent Thinks, Step by Step

This code block defines the **orchestrator** — the component that turns individual nodes into a **coherent, reliable intelligence workflow**.

Rather than relying on implicit execution or brittle scripts, the agent uses a **state-driven graph** to control how information flows, when steps run, and what depends on what.

This is what makes the agent **predictable, inspectable, and safe to operate**.

---

## Why a Graph-Based Orchestrator Matters

Most AI systems feel magical — until something goes wrong.

This agent avoids that by making execution logic explicit:

* Every step is a node
* Every dependency is visible
* Parallelism is intentional
* Nothing runs “by accident”

This structure allows leaders and engineers alike to answer:

> “What happens if we change this step?”
> “What depends on what?”
> “Where did this result come from?”

---

## StateGraph: A Single Source of Truth

The orchestrator is built on a `StateGraph`, which means:

* All nodes read from the same shared state
* All outputs are written back to that state
* No hidden side effects exist

The state becomes a **living audit trail** of the agent’s reasoning.

---

## Configuration Binding Without Coupling

Each node is wrapped so that it receives the **same configuration object**, without hard-coding parameters inside the logic.

This achieves two things:

* Behavior can be tuned without rewriting nodes
* The workflow remains stable as priorities change

This is how the agent stays flexible *without becoming fragile*.

---

## Explicit Workflow Structure

The execution flow is deliberately designed to mirror how a human analyst would work.

### Linear Where It Should Be

* Define the goal
* Create a plan
* Load the data

These steps happen in order, because they must.

---

### Parallel Where It Makes Sense

* Aggregation and sentiment analysis run in parallel
* Summarization and visualization run in parallel

This improves efficiency while keeping dependencies clear.

Parallelism is used **intentionally**, not opportunistically.

---

### Convergence Before Decisions

* Prioritization waits for both aggregation *and* sentiment
* Reporting waits for both summaries *and* visuals

This ensures no decision is made on partial information.

---

## Controlled Use of AI

The orchestrator reinforces a critical design rule:

> **LLMs are never allowed to control flow or make decisions.**

They:

* Summarize
* Explain
* Synthesize

They do *not*:

* Decide what matters
* Change priorities
* Influence execution order

The orchestrator enforces this boundary.

---

## A Single Entry Point, A Single Outcome

### `run_feedback_analysis`

This function provides a clean interface for running the entire agent:

* One call
* One configuration
* One final state

The result is a fully populated state object containing:

* All intermediate outputs
* All final insights
* All visual paths
* All errors (if any)

Nothing is hidden.

---

## Why This Matters to Leadership

From a business perspective, this orchestrator provides:

* Predictable behavior
* Repeatable results
* Clear governance boundaries
* Confidence that insights are not ad hoc

This is the difference between:

* “An AI experiment”
* And **an operational intelligence system**

---

## Architectural Takeaway

The orchestrator is the agent’s **control plane**.

It doesn’t analyze data or generate insight — it ensures that:

* The right steps happen
* In the right order
* For the right reasons
* With the right constraints

That’s what makes the entire system trustworthy.

> **This agent doesn’t just think — it follows a plan.**




In [ ]:
"""Orchestrator for Employee Feedback Intelligence Agent

Wires up all nodes into a LangGraph workflow.
"""

from typing import Optional
from langgraph.graph import StateGraph, END
from config import EmployeeFeedbackIntelligenceState, EmployeeFeedbackIntelligenceConfig
from agents.employee_feedback_intelligence.nodes import (
    goal_node,
    planning_node,
    data_loading_node,
    aggregation_node,
    sentiment_analysis_node,
    prioritization_node,
    summarization_node,
    visualization_node,
    report_generation_node
)


def create_orchestrator(config: EmployeeFeedbackIntelligenceConfig):
    """
    Create and return the orchestrator workflow.

    Args:
        config: Configuration for the agent

    Returns:
        Compiled LangGraph workflow
    """
    workflow = StateGraph(EmployeeFeedbackIntelligenceState)

    # Create wrapper functions that bind config
    def data_loading_wrapper(state):
        return data_loading_node(state, config)

    def aggregation_wrapper(state):
        return aggregation_node(state, config)

    def sentiment_analysis_wrapper(state):
        return sentiment_analysis_node(state, config)

    def prioritization_wrapper(state):
        return prioritization_node(state, config)

    def summarization_wrapper(state):
        return summarization_node(state, config)

    def visualization_wrapper(state):
        return visualization_node(state, config)

    def report_generation_wrapper(state):
        return report_generation_node(state, config)

    # Add all nodes
    workflow.add_node("goal", goal_node)
    workflow.add_node("planning", planning_node)
    workflow.add_node("data_loading", data_loading_wrapper)
    workflow.add_node("aggregation", aggregation_wrapper)
    workflow.add_node("sentiment_analysis", sentiment_analysis_wrapper)
    workflow.add_node("prioritization", prioritization_wrapper)
    workflow.add_node("summarization", summarization_wrapper)
    workflow.add_node("visualization", visualization_wrapper)
    workflow.add_node("report_generation", report_generation_wrapper)

    # Set entry point
    workflow.set_entry_point("goal")

    # Linear flow: goal -> planning -> data_loading
    workflow.add_edge("goal", "planning")
    workflow.add_edge("planning", "data_loading")

    # After data loading, run aggregation and sentiment analysis in parallel
    workflow.add_edge("data_loading", "aggregation")
    workflow.add_edge("data_loading", "sentiment_analysis")

    # Prioritization depends on both aggregation and sentiment
    workflow.add_edge("aggregation", "prioritization")
    workflow.add_edge("sentiment_analysis", "prioritization")

    # After prioritization, run summarization and visualization in parallel
    workflow.add_edge("prioritization", "summarization")
    workflow.add_edge("prioritization", "visualization")

    # Report generation depends on both summarization and visualization
    workflow.add_edge("summarization", "report_generation")
    workflow.add_edge("visualization", "report_generation")

    # End after report generation
    workflow.add_edge("report_generation", END)

    return workflow.compile()


def run_feedback_analysis(
    data_dir: Optional[str] = None,
    config: Optional[EmployeeFeedbackIntelligenceConfig] = None
) -> EmployeeFeedbackIntelligenceState:
    """
    Run the complete feedback analysis workflow.

    Args:
        data_dir: Optional directory containing feedback files (defaults to config.data_dir)
        config: Optional configuration (defaults to EmployeeFeedbackIntelligenceConfig())

    Returns:
        Final state with all analysis results
    """
    if config is None:
        config = EmployeeFeedbackIntelligenceConfig()

    # Create orchestrator
    orchestrator = create_orchestrator(config)

    # Initial state
    initial_state: EmployeeFeedbackIntelligenceState = {
        "data_dir": data_dir or config.data_dir,
        "errors": []
    }

    # Run workflow
    final_state = orchestrator.invoke(initial_state)

    return final_state

